<a href="https://colab.research.google.com/github/HuHengKai/JD/blob/master/%E5%A4%B4%E5%83%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function, division
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, GlobalAveragePooling2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from PIL import Image
import matplotlib.pyplot as plt
import os
import numpy as np

class DCGAN():
    def __init__(self):
        # 输入shape
        self.img_rows = 64
        self.img_cols = 64
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        # adam优化器
        optimizer = Adam(0.0002, 0.5)
        # 判别模型
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=['binary_crossentropy'],
            optimizer=optimizer,
            metrics=['accuracy'])
        # 生成模型
        self.generator = self.build_generator()

        # conbine是生成模型和判别模型的结合
        # 判别模型的trainable为False
        # 用于训练生成模型
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        self.discriminator.trainable = False

        valid = self.discriminator(img)

        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):
        model = Sequential()
        model.add(Dense(256 * 8 * 8, activation='relu', input_dim=self.latent_dim))  # 输入维度为100
        model.add(Reshape((8, 8, 256)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=5, padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=5, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(3, kernel_size=5, padding="same"))
        model.add(Activation("tanh"))
        model.summary()  # 打印网络参数
        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):
        model = Sequential()
        dropout = 0.5
        model.add(Conv2D(64, kernel_size=5, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(dropout))
        model.add(Conv2D(128, kernel_size=5, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(dropout))
        model.add(Conv2D(256, kernel_size=5, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(dropout))
        model.add(GlobalAveragePooling2D())
        model.add(Dense(1, activation='sigmoid'))
        model.summary()
        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):
        # 载入数据
        filename = "/content/drive/MyDrive/机器学习展示/data/"
        X_train = load_image(filename)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # --------------------- #
            #  训练判别模型
            # --------------------- #
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # 训练并计算loss
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  训练生成模型
            # ---------------------
            g_loss = self.combined.train_on_batch(noise, valid)

            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,:])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("dc_images/MeganFaces_%d.png" % epoch)
        plt.close()

def load_image(filename):
    imgs = os.listdir(filename)[:10]
    print(len(imgs))
    x_train = np.empty((imgs.__len__(), 64, 64, 3), dtype='float32')

    for i in range(len(imgs)):
        img = Image.open(filename + imgs[i])
        img = img.resize((64,64))
        img_arr = np.asarray(img, dtype='float32')
        x_train[i, :, :, :] = img_arr/127.5 - 1.

    return x_train

if __name__ == '__main__':
    if not os.path.exists("./dc_images"):
        os.makedirs("./dc_images")
    dcgan = DCGAN()
    dcgan.train(epochs=20000, batch_size=256, save_interval=50)



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 64)        4864      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32, 32, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 128)       204928    
_________________________________________________________________
batch_normalization (BatchNo (None, 16, 16, 128)       512       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 128)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 128)       0

In [1]:
import tensorflow as tf
import pathlib
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import UpSampling2D, Conv2D, Dense, BatchNormalization, LeakyReLU, Input,Reshape, MaxPooling2D, Flatten, AveragePooling2D, Conv2DTranspose
from tensorflow.keras.optimizers import Adam

In [2]:
#加载数据集
# 数据集的位置 496
avatar_img_path = "/content/drive/MyDrive/机器学习展示/data/"
from PIL import Image
import imageio
import os
import numpy as np
def load_image(filename):
    imgs = os.listdir(filename)
    x_train = np.empty((imgs.__len__(), 64, 64, 3), dtype='float32')

    for i in range(len(imgs)):
        if i%30==0:
          print(i)
        img = Image.open(filename + imgs[i])
        img = img.resize((64,64))
        img_arr = np.asarray(img, dtype='float32')
        x_train[i, :, :, :] = img_arr/127.5 - 1.

    return x_train
dataset=load_image(avatar_img_path)

0
30
60
90
120
150
180
210
240
270
300
330
360
390
420
450
480
510
540
570
600
630
660
690
720
750
780
810
840
870
900
930
960
990
1020
1050
1080
1110
1140
1170
1200
1230
1260
1290
1320
1350
1380
1410
1440
1470
1500
1530
1560
1590
1620
1650
1680
1710
1740
1770
1800
1830
1860
1890
1920
1950
1980
2010
2040
2070
2100
2130
2160
2190
2220
2250
2280
2310
2340
2370
2400
2430
2460
2490
2520
2550
2580
2610
2640
2670
2700
2730
2760
2790
2820
2850
2880
2910
2940
2970
3000
3030
3060
3090
3120
3150
3180
3210
3240
3270
3300
3330
3360
3390
3420
3450
3480
3510
3540
3570
3600
3630
3660
3690
3720
3750
3780
3810
3840
3870
3900
3930
3960
3990
4020
4050
4080
4110
4140
4170
4200
4230
4260
4290
4320
4350
4380
4410
4440
4470
4500
4530
4560
4590
4620
4650
4680
4710
4740
4770
4800
4830
4860
4890
4920
4950
4980
5010
5040
5070
5100
5130
5160
5190
5220
5250
5280
5310
5340
5370
5400
5430
5460
5490
5520
5550
5580
5610
5640
5670
5700
5730
5760
5790
5820
5850
5880
5910
5940
5970
6000
6030
6060
6090
6120
6150
6180
6210

In [ ]:
dataset.shape

In [ ]:
Batch_Size=50
Buffer_Size=20000
# 生成使用得数据集
dataset=tf.data.Dataset.from_tensor_slices(dataset).shuffle(Buffer_Size).batch(Batch_Size)

In [ ]:
def generator_model():
    model=tf.keras.Sequential()
    model.add(layers.Dense(2048,input_shape=(100,),use_bias=False))
    model.add(LeakyReLU(0.2))
    # 全连接层 2048 ->  8 * 8 * 256
    model.add(Dense(8 * 8 * 256))
    # DN层
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.2))
    # 8 * 8 * 256 -> (8,8,256)
    model.add(Reshape((8, 8, 256)))
    # 卷积层 (8,8,256) -> (8,8,128)
    model.add(Conv2D(128, kernel_size=5, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.2))
    # 反卷积层 (8,8,128) -> (16,16,128)
    model.add(Conv2DTranspose(128, kernel_size=5, strides=2, padding='same'))
    model.add(LeakyReLU(0.2))
    # 反卷积层 (16,16,128) -> (32,32,64)
    model.add(Conv2DTranspose(64, kernel_size=5, strides=2, padding='same'))
    model.add(LeakyReLU(0.2))
    # 反卷积层  (32,32,64) -> (64,64,3) = 图片
    model.add(Conv2DTranspose(3, kernel_size=5, strides=2, padding='same', activation='tanh'))
    return model

# 定义鉴别器
def discriminator_model():
    model = Sequential()
    # 卷积层
    model.add(Conv2D(64, kernel_size=5, padding='valid',input_shape = (64,64,3)))
    # BN层
    model.add(BatchNormalization())
    # 激活层
    model.add(LeakyReLU(0.2))
    # 平均池化层
    model.add(AveragePooling2D(pool_size=2))
    # 卷积层
    model.add(Conv2D(128, kernel_size=3, padding='valid'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.2))
    model.add(AveragePooling2D(pool_size=2))
    model.add(Conv2D(256, kernel_size=3, padding='valid'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.2))
    model.add(AveragePooling2D(pool_size=2))
    # 将输入展平
    model.add(Flatten())
    # 全连接层
    model.add(Dense(1024))
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.2))
    # 最终输出1(true img) 0(fake img)的概率大小
    model.add(Dense(1, activation='sigmoid'))
    return model
# 定义损失,带有logit得交叉损失
cross_entropy=keras.losses.BinaryCrossentropy(from_logits=True)
# 定义鉴别器损失
def discriminator_loss(real_out,fake_out):
  real_loss=cross_entropy(tf.ones_like(real_out),real_out)
  fake_loss=cross_entropy(tf.zeros_like(fake_out),fake_out)
  return real_loss+fake_loss
# 定义生成器损失
def generator_loss(fake_out):
    fake_loss = cross_entropy(tf.ones_like(fake_out), fake_out)
    return fake_loss
# 寻优标准
generator_opt=tf.keras.optimizers.Adam(0.0001)
discriminator_opt=tf.keras.optimizers.Adam(0.0001)
Epochs=100
input_dim=100
num_exp_to_generate=16
# shape=(16, 100)
seed=tf.random.normal([num_exp_to_generate,input_dim])
# 模型实例化
generator=generator_model()
discriminator=discriminator_model()

In [ ]:
def train_step(images):
    noise=tf.random.normal([Batch_Size,input_dim])
    with tf.GradientTape() as gen_tape,tf.GradientTape() as dis_tape:
        real_out=discriminator(images)
        gen_img =generator(noise)
        fake_out=discriminator(gen_img)
        dis_loss=discriminator_loss(real_out,fake_out)
        gen_loss=generator_loss(fake_out)
 
    gen_gard=gen_tape.gradient(gen_loss,generator.trainable_variables)
    dis_gard = dis_tape.gradient(dis_loss, discriminator.trainable_variables)
    discriminator_opt.apply_gradients(zip(dis_gard,discriminator.trainable_variables))
    generator_opt.apply_gradients(zip(gen_gard, generator.trainable_variables))
 
def genrate_plot_image(gen_model,test_noise):
    pre_images=gen_model(test_noise,training=False)
    fig=plt.figure(figsize=(4,4))
    print(pre_images.shape[0])
    for i in range(pre_images.shape[0]):
        plt.subplot(4,4,i+1)
        plt.imshow((pre_images[i,:,:,0]+1)/2*255.)
        plt.axis('off')
    plt.show()
 
def train(dataset,epochs):
    for epoch in range(epochs):
        for image_batch in dataset:
            train_step(image_batch)
        print(epoch)
        genrate_plot_image(generator,seed)
 
if __name__ == '__main__':
    train(dataset,100)